In [17]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from numpy import pi

qreg_q = QuantumRegister(3,'q')
maxD = QuantumCircuit(qreg_q, name='maxD')

maxD.x(qreg_q[2])

maxD.ccx(qreg_q[2], qreg_q[1], qreg_q[0])
maxD.cx(qreg_q[0], qreg_q[1])

maxD.to_gate()
maxD.draw()

┌───┐     
q_0: ─────┤ X ├──■──
          └─┬─┘┌─┴─┐
q_1: ───────■──┤ X ├
     ┌───┐  │  └───┘
q_2: ┤ X ├──■───────
     └───┘

In [14]:
def runCircuit(circuit):
    simulator = Aer.get_backend('qasm_simulator')
    res = execute(circuit, backend=simulator, shots=1).result()
    counts = res.get_counts()
    return counts

In [15]:
def testCircuitInint(circToTest, vec): 
    tester = QuantumCircuit(3,3)
    for i, val in enumerate(reversed(vec)):
#         print(i)
        state_vec = [1-int(val) , int(val)] 
        tester.initialize(state_vec, i)
    tester.barrier()
    tester.append(circToTest, [0,1,2])
    tester.barrier()
    tester.measure([0,1,2], [0,1,2])
    tester.draw()
    return tester

In [16]:
# vec[0] - hot, vec[1] - cold, vec[2] = daemon
# We expect the daemon to flip the hot
truthTable = [
    '001',
    '011',
    '111',
    '101',
    '000',
    '010',
    '110',
    '100'
]

for vec in truthTable:
    circ = testCircuitInint(maxD, vec)
    counts = runCircuit(circ)
    print('{} -> {}'.format(vec,list(counts.keys())[0]))
    
circ.draw()

001 -> 101
011 -> 111
111 -> 011
101 -> 001
000 -> 100
010 -> 110
110 -> 010
100 -> 000


┌─────────────────┐ ░ ┌───────┐ ░ ┌─┐      
q_0: ┤ initialize(1,0) ├─░─┤0      ├─░─┤M├──────
     ├─────────────────┤ ░ │       │ ░ └╥┘┌─┐   
q_1: ┤ initialize(1,0) ├─░─┤1 maxD ├─░──╫─┤M├───
     ├─────────────────┤ ░ │       │ ░  ║ └╥┘┌─┐
q_2: ┤ initialize(0,1) ├─░─┤2      ├─░──╫──╫─┤M├
     └─────────────────┘ ░ └───────┘ ░  ║  ║ └╥┘
c: 3/═══════════════════════════════════╩══╩══╩═
                                        0  1  2